# 2.5. 用插件和智能体增强答疑机器人的能力

## 🚅 前言
在前面的课程中，你已经能够通过优化提示词，为大模型提前设定回答问题的角色、格式等，或者通过优化RAG过程中的文档解析/切片、图片的深层次理解、标题改写等方法，提高了大模型检索知识库中有效、正确信息的能力；但是大模型有一定的局限性，如果你想要大模型在帮你制定好旅游攻略后再帮你预定酒店，即使加入了信息检索知识库等能力，大模型依然无法做到，这是因为大模型不能理解和执行用户的意图，无法根据用户的意图，自动的调用外部工具来解决问题。

您可以使用智能体（Agent）应用来解决这些问题，以大模型为基础，根据业务场景集成特定外部工具/插件，以此来增强大模型的能力。

## 🍁 课程目标
学完本课程后，你将能够：

* 了解Agent的原理；
* 了解 Assistant API 的基本概念以及运行机制；
* 掌握 Assistant API 的基础编码方法；
* 能够动手使用Assistant API构建Agent应用；


## 📖 课程目录

- [1. 智能体（Agent）](#🤖-1-智能体（agent）)
    - [1.1. 什么是智能体（Agent）](#11-什么是智能体（agent）)
    - [1.2. 应用场景](#12-应用场景)
    - [1.3. 运行机制](#13-运行机制)
- [2. 如何构建Agent](#🛠️-2-如何构建agent)
    - [2.1. 输出多模态内容](#21-输出多模态内容)
    - [2.2. 构建你的个性化语音助手](#22-构建你的个性化语音助手)
- [3. 扩展阅读：Multi-Agent](#🤼‍♀️-3-扩展阅读：multi-agent)



## 🤖 1. 智能体（Agent）

### 1.1. 什么是智能体（Agent）
Agent是大模型为基础，能够让大模型自主规划，自动调用外部插件，辅助大模型使用工具与外部世界进行交互的应用。

<div align="center">
<img src="https://img.alicdn.com/imgextra/i2/O1CN01pKxr9I1JMv0KOidGH_!!6000000001015-0-tps-1532-694.jpg" alt="image" style="width:800px;height:350px;">
</div>

### 1.2. 应用场景

智能体作为一种设计模式，在知识问答、工业自动化等场景中均有应用。本文主要介绍知识问答，尤其是具备知识库增强的应用场景中，调用外部工具可以极大地扩展系统的能力，使其不仅仅局限于文本交互，还能够处理更复杂的用户需求和场景。

#### 1.2.1. 生成多模态内容
**内容输出的意图识别**：Agent能够分析用户的查询意图，并根据这些意图确定最合适的输出形式。例如，如果用户询问“明天的天气如何？”，Agent不仅能够理解这是一个关于天气的查询，还能够判断出用户可能需要图像或视频形式的天气预测。  
**构建场景**：Agent可以根据用户的查询构建相应的场景，并输出相应的语音、视频或图像内容。例如，对于一个旅游查询，Agent可以生成一个旅游目的地的360度视频，让用户有身临其境的感觉。   
**输出多样化内容**：Agent可以调用图像生成API来创建定制的图像，如根据用户描述生成艺术作品，或者调用语音合成API来朗读文本内容，提供更丰富的用户体验。

#### 1.2.2. 对接业务系统
**工具调用的意图识别**： Agent能够识别用户需求背后的工具调用意图。例如，如果用户想要预订机票，Agent可以识别出这一需求，并调用相应的预订系统API来完成操作。   
**调用工具或API**： Agent可以与各种外部系统对接，如日历应用、邮件服务、在线支付平台等，通过调用这些工具或API来执行用户的请求。例如，Agent可以帮助用户设置提醒、发送邮件或完成在线购物。

#### 1.2.3. 自我反思纠错
**参数获取**：在用户给出的信息不完整时，Agent可以通过多轮对话来获取缺失的参数，这要求Agent能够识别出哪些信息是必需的，并且能够以一种自然的方式引导用户提供这些信息。例如，如果用户说“我想预订酒店”，但没有提供日期和地点，Agent可以主动询问用户“您想预订哪天的酒店，以及您将在哪里旅行？”   

**自主推理**：Agent可以基于已有的信息进行推理，以确定需要哪些额外信息，这涉及到对上下文的理解以及对可能的下一步行动的预测。例如，如果用户提到“我需要为我的旅程购买保险”，Agent可以推断出需要知道旅程的日期、目的地和用户的年龄来提供保险选项。    

**自主纠错**：在对话过程中，Agent可以监控对话的上下文，如果发现用户的信息有误或不完整，它可以主动提出问题或建议来纠正错误或补充信息。例如用户提到“我想买一张去纽约的机票，明天的”，Agent可以纠错用户的问题，如“明天没有直飞纽约的航班，最近的航班是后天的”

**主动询问**：Agent可以通过主动询问来引导用户提供更多信息。例如，如果用户说“我想看一部科幻电影”，Agent可以询问“您想看最近上映的还是经典的科幻电影？您对电影的时长或导演有偏好吗？   

通过这些高级功能，Agent问答系统能够提供更加动态、个性化和高效的服务。它们能够更好地理解用户的意图，提供更丰富的交互体验，并在需要时主动获取更多信息以完成任务。这种系统不仅提高了用户体验，还能够在各种场景中提供实际帮助，从而在客户服务、个人助理、在线教育等领域得到广泛应用。


### 1.3. 运行机制
Assistant 接受用户输入，并由内置的大语言模型判断是否需要调用工具。

- 如果需要调用工具，大语言模型会选择合适的工具，工具返回结果和用户内容合并后再次输入到大语言模型，由大语言模型生成内容并输出。

- 如果无需调用工具，大语言模型将直接生成内容并输出。

<div align="center">
<img src="https://img.alicdn.com/imgextra/i4/O1CN01fUJjoZ1a0KwdHEEo6_!!6000000003267-0-tps-828-752.jpg" alt="image" style="width:400px;height:400px;">
</div>

下面将以传统问答系统、加入大模型的问答系统和加入Agent的问答系统之间的区别与运行原理，帮助大家理解Agent应用：

#### 1.3.1. 传统问答系统

传统问答系统（FAQ-based systems）是一种基于预先定义的问题和答案对的系统，通常用于提供快速的信息检索和响应。这些系统的设计和功能相对简单，主要依赖于关键词匹配和固定的答案库。
- 静态知识库：系统包含一个预先定义好的问题和答案库，这些问答对通常在系统部署前就已经设定好。
- 关键词匹配：用户提出的问题通过关键词匹配来查找答案。系统会分析用户输入的关键词，然后在知识库中搜索匹配的答案。
- 有限的交互：用户和系统的交互通常非常有限，用户提出问题，系统返回一个预设的答案。
- 缺乏上下文理解：系统通常不理解问题的上下文或复杂性，因此可能无法处理含糊或复杂的问题。
- 缺乏个性化：系统不会根据用户的历史交互或偏好来个性化回答。
- 维护成本：随着问答库的增长，维护和更新这些系统可能变得复杂和耗时，依赖人工更新：新问题的答案需要人工添加到知识库中。
- 缺乏灵活性：系统通常无法处理未预见的问题或需要创造性回答的问题。
  
<div align="center">
<img src="https://img.alicdn.com/imgextra/i4/O1CN01yZZNzH1DoVA9p6B20_!!6000000000263-0-tps-1482-550.jpg" alt="image" style="width:800px;height:300px;">
</div>

#### 1.3.2. 加入大模型的问答系统

加入大模型的问答系统是一种更先进的技术，它利用了大规模预训练语言模型来理解和生成自然语言。这些系统通常能够处理更复杂的语言结构和上下文，提供更准确和个性化的回答。
- 上下文理解：大模型通过在大量文本数据上的预训练，能够理解语言的复杂性和上下文，从而提供更准确的回答。
- 个性化回答：系统可以根据用户的历史交互和偏好来个性化回答，支持更复杂的多轮对话，能够跟踪对话的上下文并在对话过程中提供连贯的回答。
- 自我学习和适应：大模型可以通过持续学习来改进其性能。
- 私有知识库：可以检索知识库中的答案，能够生成新的回答，这使得系统更加灵活。
- 可扩展性：大模型可以轻松集成到不同的平台和应用中，提供跨平台的问答服务，并且大模型可以通过微调来更新其知识库，以反映最新的信息和数据。
- 安全性和合规性：大模型可以被训练以遵守特定的安全和合规性标准。
- 多语言支持：许多大模型支持多种语言，使得问答系统可以服务于全球用户。

<div align="center">
<img src="https://img.alicdn.com/imgextra/i2/O1CN01zibyXW1jXcGWTnQBU_!!6000000004558-0-tps-1286-502.jpg" alt="image" style="width:700px;height:280px;">
</div>

#### 1.3.3. 加入Agent的问答系统

在Agent问答系统中，Agent作为一个自主的智能实体，能够理解用户的查询意图，并根据这些意图调用相应的工具或API来完成任务。Agent可以自主规划、自动调用插件或者工具，辅助大模型使用工具与外部世界进行交互。
与传统问答系统相比，Agent问答系统的主要优势包括：
- 多模态内容生成：Agent可以识别内容输出的意图，并构建场景来输出语音、视频、图像等多种内容，提供更丰富的交互体验。
- 系统对接：Agent能够识别工具调用的意图，调用工具或API来执行特定的任务。
- 自我反思能力：当用户提供的信息不完整时，Agent可以通过构建多轮对话、自主推理、自主纠错，主动询问等方式获取所需的参数。
- 智能规划：Agent可以根据用户的输入和其他Agent的功能，自主调用合适的Agent来完成任务。

<div align="center">  
<img src="https://img.alicdn.com/imgextra/i4/O1CN01FAiU1K1Dscj9YALce_!!6000000000272-0-tps-1472-734.jpg" alt="image" style="width:800px;height:350px;">
</div>


## 🛠️ 2. 如何构建Agent
### 2.1. 输出多模态内容

具体操作步骤如下，这里我们将带大家实现一个绘画助手，

#### 💻 计算环境准备
##### 🔧 安装依赖

In [ ]:
! pip ins! pip install -r requirements.txt

#### 步骤 1. 导入依赖

In [46]:
import dashscope
import json
from http import HTTPStatus

#### 步骤 2. 封装调用状态查询函数

In [47]:
def check_status(component, operation):
    if component.status_code == HTTPStatus.OK:
        print(f"{operation} 成功。")
        return True
    else:
        print(f"{operation} 失败。状态码：{component.status_code}，错误码：{component.code}，错误信息：{component.message}")
        return False

#### 步骤 3. 创建绘画助手Assistant
具体到我们的案例，目标是构建一个专注于绘画的 Assistant。基于文生图工具对语言理解能力的较高要求，我们选用了通义千问-Max作为文本推理模型，以此强化 Assistant 的语义理解和文本生成能力。

智能体的配置细节，包括其名称、功能描述及指令，均在随附的代码片段中有明确展示。为了丰富智能体的功能性和实用性，我们集成了两个官方预置插件：

- 夸克搜索，旨在帮助智能体获取关于宠物猫的广泛文本信息；

- 图片生成，确保智能体能够根据接收到的文字描述，自动生成相应的图像内容。

In [48]:
import dashscope

# 使用 Qwen-Max 创建一个新的绘画助手
painting_assistant = dashscope.Assistants.create(
    model='qwen-max' ,  # model大语言模型名，用于为智能体配置大模型
    name='Art Maestro', # name智能体名称，用于区别智能体的名称
    description='用于绘画和艺术知识的AI助手', # description智能体的功能描述
    instructions='''提供绘画技巧、艺术史和创意指导的信息。
    使用工具进行研究和生成图像。''', # 由自然语言构成的指令，用于定义智能体的角色和任务
    tools=[
        {'type': 'quark_search', 'description': '用于研究艺术主题'},
        {'type': 'text_to_image', 'description': '用于创建视觉示例'}
    ]
) # tools为智能体配置的工具列表

# 打印助手的ID以确认创建成功
if not check_status(painting_assistant, "助手创建"):
    exit()
print(f"绘画助手 'Art Maestro' 创建成功，ID：{painting_assistant.id}")

助手创建 成功。
绘画助手 'Art Maestro' 创建成功，ID：asst_c0b4cb8b-3295-41bd-8a7c-39df66e50902


#### 步骤 4. 创建Thread
Thread（线程）是 Assistant API 中的一个关键概念，它代表了一个持续的对话上下文。

Thread 允许您在用户启动新对话时创建一个会话管理线程。Assistant 可以通过 Thread 理解整个对话的上下文，从而提供更加连贯和相关的回应。在绘画助手的场景中，Thread 可以跟踪用户的初始请求、Assistant 的初步建议、用户的反馈，以及最终的绘画结果，形成一个完整的创作过程。这确保了整个创作过程的连贯性和可追溯性。

In [49]:
from http import HTTPStatus
import dashscope

# 创建一个新的空线程
thread = dashscope.Threads.create()

if not check_status(thread, "线程创建"):
    exit()

# 注意：这个空线程现在可以用于保持你绘画项目讨论的上下文，
# 包括任何关于布偶猫或其他绘画主题的未来消息。

线程创建 成功。


#### 步骤 5. 添加 Message 到 Thread

您输入的内容将通过 Message 对象传递。 Assistant API支持向单一 Thread 发送一个或多个消息。创建 Message 时，您需要考虑以下参数：

- Thread 唯一编号thread_id

- 消息内容content

尽管 Thread 可接收的 Token 数量无硬性限制，但实际传递给大模型的 Token 数量需符合该模型的最大输入长度限制，具体参照各通义千问系列模型的官方文档中关于上下文长度的说明。在我们的场景中，您将通过 Message 在 Thread 中发出第一条消息：“请帮我画一只布偶猫的图片”。您需要创建一个 Message 类，详细参数设置均在随附的代码片段中。

In [50]:
# 创建一条消息，告诉助手需要做什么？
message = dashscope.Messages.create(thread.id, content='请帮我画一张蜡笔小新的漫画。')
if not check_status(message, "消息创建"):
    exit()

消息创建 成功。


####  步骤 6. 创建Run并执行
用户将消息分配至特定 Thread 后，可以通过启动一个运行（Run）来激活预设的助手（Assistant）。该助手会以线程内的所有消息为背景，利用指定的模型和可用的插件，智能地回应用户的问题，并将生成的答案插入到线程的消息序列中。

在本场景中，您需执行以下步骤：

- 初始化一个运行对象，以驱动绘画助手，传递线程ID（thread.id）和助手ID（assistant.id）。

- 使用运行对象的等待方法（Run.wait），直至执行完毕。

In [ ]:
run = dashscope.Runs.create(thread.id, assistant_id=painting_assistant.id)

if not check_status(run, "运行创建"):
    exit()

print("等待助手处理请求...")
run = dashscope.Runs.wait(run.id, thread_id=thread.id)

if check_status(run, "运行完成"):
    print(f"运行完成，状态：{run.status}")
else:
    print("运行未完成。")
    exit()

#### 步骤 7. 返回结果
这一步通过消息列表方法（Messages.list），检索助手绘制的宠物猫图片。这一系列操作确保了助手从接收问题到输出结果的自动化处理流程。

In [52]:
messages = dashscope.Messages.list(thread.id)

if check_status(messages, "消息检索"):
    if messages.data:
        # 显示最后一条消息的内容（助手的响应）
        last_message = messages.data[0]
        print("\n助手的回应：")
        print(json.dumps(last_message, ensure_ascii=False, default=lambda o: o.__dict__, sort_keys=True, indent=4))
    else:
        print("在线程中未找到消息。")
else:
    print("未能检索到助手的响应。")

消息检索 成功。

助手的回应：
{
    "assistant_id": "asst_c0b4cb8b-3295-41bd-8a7c-39df66e50902",
    "completed_at": 1727678566561,
    "content": [
        {
            "text": {
                "annotations": [],
                "value": "好的，我已经为您创作了一幅布偶猫的画。请欣赏：\n\n![布偶猫的画](https://dashscope-result-sh.oss-cn-shanghai.aliyuncs.com/1d/36/20240930/29c02cfd/8f1b0a44-2d65-4758-baa5-240a6e279357-1.png?Expires=1727764956&OSSAccessKeyId=LTAI5tQZd8AEcZX6KZV4G8qL&Signature=ns7sOPhCOTk6K1kE%2BYcvb4OO%2Fvg%3D)\n\n希望您喜欢这幅作品。"
            },
            "type": "text"
        }
    ],
    "created_at": 1727678557892,
    "file_ids": [],
    "id": "message_5b118403-cefe-4f8f-9909-a5056691aab0",
    "incomplete_at": null,
    "incomplete_details": null,
    "metadata": {},
    "name": "",
    "object": "thread.message",
    "plugin_call": {},
    "role": "assistant",
    "run_id": "run_504d0eb1-3fbe-4cd4-97ba-cc953434062d",
    "status": "",
    "thread_id": "thread_4431005c-dcd6-4be0-bd48-1d04f2f450d4",
    "t

### 2.2. 构建你的个性化语音助手

下面这个案例将结合私有知识库，结合本地自定义插件来构建agent。本案例基于百炼上已经创建的RAG应用， [操作方法请参考最佳实践-基于RAG的官方文档助手](https://help.aliyun.com/zh/model-studio/user-guide/rag-official-document-assistant?spm=a2c4g.11186623.0.0.2f251078R4BNzj)，结合本地的语音合成工具，帮助大模型输出文本语言的同时，也可以输出音频数据；你可以根据你业务本身的需要，按照下列方法构建Function，来实现更加定制化的应用。流程如下：

<div align="center">
<img src="https://img.alicdn.com/imgextra/i4/O1CN01W1jx4b1sJGOZL0lU1_!!6000000005745-0-tps-1046-786.jpg" alt="image" style="width:500px;height:360px;">
</div>

前期准备：
- 已开通百炼服务：[产品开通](https://help.aliyun.com/zh/model-studio/getting-started/activate-alibaba-cloud-model-studio?spm=a2c4g.11186623.0.0.2f252a02dyN9ui)。

- 已创建百炼API_KEY：[获取API-KEY](https://help.aliyun.com/zh/model-studio/developer-reference/get-api-key?spm=a2c4g.11186623.0.0.2f252a02dyN9ui)

- 已创建百炼知识库，获取知识库ID： [创建知识库](https://help.aliyun.com/zh/model-studio/user-guide/rag-knowledge-base?spm=a2c4g.11186623.0.0.193bfa89ZwRJCv)。

- 获取百炼工作空间ID：[获取工作空间ID](https://help.aliyun.com/zh/model-studio/developer-reference/use-workspace?spm=a2c4g.11186623.0.0.79a535a2fnDIyz&scm=20140722.S_help@@%E6%96%87%E6%A1%A3@@2587495@@2.S_llmOS0.ID_698593-RL_%E7%99%BE%E7%82%BC%E5%B7%A5%E4%BD%9C%E7%A9%BA%E9%97%B4ID-LOC_chat~DAS~llm-OR_ser-PAR1_212bee0f17287002915655784d0095-V_3-P0_0)。

- 已获取阿里云的AccessKey和AccessKeySecret：[获取访问密钥](https://help.aliyun.com/zh/voice-navigator/developer-reference/get-accesskey?spm=a2c4g.11186623.0.0.79a535a2fnDIyz&scm=20140722.S_help@@%E6%96%87%E6%A1%A3@@155052@@1.S_llmOS0.ID_279330-RL_%E6%82%A8%E7%9A%84%E9%98%BF%E9%87%8C%E4%BA%91%E8%AE%BF%E9%97%AE%E5%AF%86%E9%92%A5ID-LOC_chat~DAS~llm-OR_ser-PAR1_213ea45a17287004278671508d0098-V_3-P0_0)

- 已安装最新版SDK：[安装SDK](https://help.aliyun.com/zh/model-studio/developer-reference/install-sdk?spm=a2c4g.11186623.0.0.2f252a02dyN9ui)。

#### 步骤 1. 导入依赖

In [2]:
import dashscope
import os
from dashscope.audio.tts_v2 import *
from dashscope import Application,Assistants,Messages, Runs, Threads
import IPython
import json
from http import HTTPStatus

from alibabacloud_bailian20231229.client import Client as bailian20231229Client
from alibabacloud_tea_openapi import models as open_api_models
from alibabacloud_bailian20231229 import models as bailian_20231229_models
from alibabacloud_tea_util import models as util_models
from alibabacloud_tea_util.client import Client as UtilClient

#### 步骤 2. 创建工具函数
用户可以自行准备各类待使用的私有API或者Function能力，用于处理本地调用的请求。

In [4]:
import os
from dotenv import load_dotenv

## for MacOS users
filePath = os.path.abspath(os.path.expanduser(os.path.expandvars("~/.zshrc")))
load_dotenv(filePath)

# dashscope.api_key = os.getenv("DASHSCOPE_API_KEY"),  # 替换为您的API Key

def get_audio(message,model='cosyvoice-v1',voice="longxiaochun"):
    synthesizer = SpeechSynthesizer(model=model, voice=voice)
    audio = synthesizer.call(message)
    # print('requestId: ', synthesizer.get_last_request_id())
    with open('out.mp3', 'wb') as f:
        f.write(audio)
    audio_play = IPython.display.Audio(audio)
    display(audio_play)
    return '音频合成成功'
get_audio('今天天气怎么样呀')

'音频合成成功'

#### 步骤 3. 调用百炼应用的RAG应用

In [113]:
def call_rag_app(prompt='内容开发工程师的工作职责是什么'):
    response = Application.call(app_id='c385d3e902d049d29e749bedfb4c4259',
                                prompt=prompt)

    if response.status_code != HTTPStatus.OK:
        print('request_id=%s, code=%s, message=%s\n' % (response.request_id, response.status_code, response.message))
        answer = response.status_code
    else:
        print('request_id=%s\n output=%s\n usage=%s\n' % (response.request_id, response.output, response.usage))
        answer = response.output["text"]

    return answer

#### 4. 基于Assistant API，实现Function调用
实现在通过检索文档的基础上，完成各类本地function调用的能力，本案例是实现文本转音频的输出，对工具的描述如下：
- name: 文本生成语音，也就是前面封装好的函数get_audio，将在智能体中被调用。
- description: 提供了该工具的描述，帮助用户理解其用途。
- parameters: 定义了函数的参数，这里是大模型的输出 message，且这是必传函数

In [114]:
def create_assistant():
    assistant = Assistants.create(
        model="qwen-max",
        name='smart helper',
        description='一个智能助手，可以通过用户诉求，调用语音合成插件，将你的回答输出成语音。',
        instructions='你是一个语音合成助手，需要将结合用户的输入，并使用语音合成插件将大模型的回答输出音频',
        tools=[
            {
                'type': 'function',
                'function': {
                    'name': '文本生成语音',
                    'description': '将大模型生成的文本合成为音频',
                    'parameters': {
                        'type': 'object',
                        'properties': {
                            'message': {
                                'type': 'str',
                                'description': '大模型生成的文本，将会被转换为音频'
                            },
                        },
                        'required': ['message']},
                },
            },
        ]
        
    )

    return assistant

send_message 模块用来传递message给assistant，

In [115]:
function_mapper = {
    "文本生成语音": get_audio
}
def send_message(assistant, message=''):
    print(f"Query: {message}")

    # create a thread.
    thread = Threads.create()
    print(thread)

# create a message.
    message = Messages.create(thread.id, content=message)
    print(message)

    run = Runs.create(thread.id, assistant_id=assistant.id)
    print(run)

# # get run statue# run_status = Runs.get(run.id, thread_id=thread.id)# print(run_status)# wait for run completed or requires_action
    run_status = Runs.wait(run.id, thread_id=thread.id)
    print('插件调用前：')
    print(run_status)
    if run_status.status == 'failed':
        print('run failed:')
        print(run_status.last_error)
    if run_status.required_action:

        f = run_status.required_action.submit_tool_outputs.tool_calls[0].function
        func_name = f['name']
        param = json.loads(f['arguments'])
        print(f)
        if func_name in function_mapper:
            output = function_mapper[func_name](**param)
        else:
            output = ""

        tool_outputs = [{
            'output':
                output
        }]


        run = Runs.submit_tool_outputs(run.id,
                                       thread_id=thread.id,
                                       tool_outputs=tool_outputs)

        # should wait for run completed
        run_status = Runs.wait(run.id, thread_id=thread.id)
        print(run_status)

    run_status = Runs.get(run.id, thread_id=thread.id)
# print(run_status)# verify_status_code(run_status)# get the thread messages.
    msgs = Messages.list(thread.id)
    print("运行结果:")
    for message in msgs['data'][::-1]:
        print("content: ", message['content'][0]['text']['value'])
    print("\n")

call_rag_app 模块是之前创建的应用，调用send_message函数返回大模型的输出

In [116]:
assistant = create_assistant()
# answer = call_rag_app(prompt='模型训练？') # example 1
answer = call_rag_app(prompt='内容开发工程师的工作职责是什么，请用简洁的语言描述一下')

send_message(assistant=assistant, message=answer)
IPython.display.Audio('out.mp3')

request_id=3c7076ac-ef1c-96b9-8e17-92076f89e4bd
 output={"text": "内容开发工程师的核心职责是结合教育理论与技术实践，研发高质量的教学内容以促进学习者成长。具体包括：研究教育技术趋势与学习理论，开发教材与课程，优化及更新内容，确保教育技术平台的兼容性，参与教师培训，进行数据分析评估，以及跨部门合作，从内容创作到发布全程参与并保证质量与效果。", "finish_reason": "stop", "session_id": "4eabb27c5e4346a1bab8c1e928f0dd34", "thoughts": null, "doc_references": null}
 usage={"models": [{"model_id": "qwen-max", "input_tokens": 1599, "output_tokens": 78}]}

Query: 内容开发工程师的核心职责是结合教育理论与技术实践，研发高质量的教学内容以促进学习者成长。具体包括：研究教育技术趋势与学习理论，开发教材与课程，优化及更新内容，确保教育技术平台的兼容性，参与教师培训，进行数据分析评估，以及跨部门合作，从内容创作到发布全程参与并保证质量与效果。
{'id': 'thread_189ee1b8-80a8-4e66-a0b3-87aa091ea38f', 'object': 'thread', 'created_at': 1727685172990, 'metadata': {}, 'request_id': '5109cbe3-decc-984c-981b-53851e7881c5', 'status_code': 200}
{'content': [{'text': {'annotations': [], 'value': '内容开发工程师的核心职责是结合教育理论与技术实践，研发高质量的教学内容以促进学习者成长。具体包括：研究教育技术趋势与学习理论，开发教材与课程，优化及更新内容，确保教育技术平台的兼容性，参与教师培训，进行数据分析评估，以及跨部门合作，从内容创作到发布全程参与并保证质量与效果。'}, 'type': 'text'}], 'id': 'message_02f541c4-3265-427b-bf70

{
    "assistant_id": "asst_ca4a4390-0abf-4417-9128-e038e8b49dca",
    "cancelled_at": null,
    "completed_at": 1727685179945,
    "created_at": 1727685173306,
    "expires_at": null,
    "failed_at": null,
    "file_ids": [],
    "id": "run_ba72cc8f-8637-4767-bed4-5fc23f1ec197",
    "instructions": "\u4f60\u662f\u4e00\u4e2a\u8bed\u97f3\u5408\u6210\u52a9\u624b\uff0c\u9700\u8981\u5c06\u7ed3\u5408\u7528\u6237\u7684\u8f93\u5165\uff0c\u5e76\u4f7f\u7528\u8bed\u97f3\u5408\u6210\u63d2\u4ef6\u5c06\u5927\u6a21\u578b\u7684\u56de\u7b54\u8f93\u51fa\u97f3\u9891\uff0c\u4f7f\u7528\u97f3\u9891\u64ad\u653e\u63d2\u4ef6\u5c06\u97f3\u9891\u663e\u793a\u51fa\u6765\uff0c\u8bf7\u4f60\u6309\u7167\u4ee5\u4e0b\u987a\u5e8f\u8c03\u7528\u63d2\u4ef6\u3002\u8bf7\u6309\u4f18\u5148\u7ea7\u4f9d\u6b21\u67e5\u8be2\u4e0b\u5217\u60c5\u51b5\uff1a1. \u5148\u8c03\u7528\u8bed\u97f3\u5408\u6210\u63d2\u4ef6\u5c06\u5927\u6a21\u578b\u7684\u56de\u7b54\u8f93\u51fa\u97f3\u9891\u30022. \u518d\u8c03\u7528\u97f3\u9891\u64ad\u62a5\u63d2\

## 🤼‍♀️ 3. 扩展阅读：Multi-Agent

上述讲解的是单一智能体的应用，如果想要构建Multi-Agent多智能体，可以阅读下面内容。

### Multi-Agent
用户可以引入多个扮演不同角色的LLM Agents参与到实际的任务中，Agents之间会进行竞争和协作等多种形式的动态交互，进而产生惊人的群体智能效果。复旦NLP团队在[Agent综述论文中-《The Rise and Potential of Large Language Model Based Agents: A Survey》](https://arxiv.org/pdf/2309.07864.pdf)提到，多代理系统（Multi-Agent System）是分布式人工智能领域中的热门研究问题之一，它主要关注代理们如何有效地协调并协作解决问题，多代理系统一般可以分成协作型互动、对抗型互动两种。

<div align="center">
<img src="https://img.alicdn.com/imgextra/i1/O1CN01WRSZhX1QJthdGpdWw_!!6000000001956-0-tps-1102-436.jpg" alt="image" style="width:750px;height:300px;">
</div>

**协作型互动**   
协作型Agent应用是实际应用中应用最广泛的模式，它可以有效提高任务效率，改善集体决策，解决单个智能体无法独立解决的复杂现实问题，最终实现协同互补的目标。协作型互动又分为无序合作与有序合作。
- 当所有代理自由地表达自己的观点、看法，以一种没有顺序的方式进行合作时，称为无序合作。
- 当所有代理遵循一定的规则，例如以流水线的形式逐一发表自己的观点时，整个合作过程井然有序，称为有序合作。
  
**对抗型互动**  
Agent应用之间以一种针锋相对（tit for tat）的方式进行互动。通过竞争、谈判、辩论的形式，代理抛弃原先可能错误的信念，对自己的行为或者推理过程进行有意义的反思，最终带来整个系统响应质量的提升。

#### Agent驱动的虚拟软件公司-ChatDev
ChatDev是全流程自动化软件开发框架，ChatDev可以比拟为一个由多智能体协作运营的虚拟软件公司，在“人类”用户指定一个具体的任务需求时，不同角色的智能体能够进行交互式协同，生产出一个完整软件（包括源代码、环境依赖说明书、用户手册等）。ChatDev的主要目标是提供一个基于大语言模型的易使用、高度定制化并且可扩展的框架，是研究群体智能的理想场景。   

ChatDev借鉴软件工程瀑布模型的思想，将其分为软件设计、系统开发、集成测试、文档编制四个主要环节。之后通过软件开发瀑布模型进一步拆解，形成由原子任务构成的交流链，整条链路可以视为软件生产线，链路中每个子任务通过充当专业角色（产品设计官、Python程序员、测试工程师）的智能体进行对话式信息交互和决策，驱动整条链路能够自主的进行需求分析、系统开发、集成测试、GUI构建、文档写作等全流程开发工作。   

<div align="center">
<img src="https://img.alicdn.com/imgextra/i4/O1CN01FucMo01slex37BpcP_!!6000000005807-0-tps-1220-1090.jpg" alt="image" style="width:500px;height:400px;">
</div>

驱动智能体交流对话的主要机制为：角色专业化（Role Specialization）、记忆流（Memory Stream）、自反思（Self-Reflection）:

- 角色专业化通过角色扮演机制（Role-Playing）确保每个智能体各司其职；每个角色都有其特定的目标和行为准则，这些准则指导智能体如何理解和回应用户的需求，帮助智能体在特定领域内提供更专业、更有针对性的服务。
- 记忆流通过将历史对话进行呈现，保证上下文感知的对话过程，捕捉用户在当前会话中提到的信息或者是记住用户的历史偏好和行为模式，并动态地对对话历史信息进行汇总和决策。
- 自反思机制在对话没有自动触发结束协议时生效，通过对自己行为和决策过程进行内部评估，分析智能体的响应是否达到了预期，通过自反思，智能体可以学习如何改进自己的行为，以更好地满足用户的需求。自反思还可以包括对交流策略的调整，以及对用户反馈的响应，从而使智能体能够不断进化和适应。

<div align="center">
<img src="https://img.alicdn.com/imgextra/i3/O1CN011P5glB1hCt6o6smB0_!!6000000004242-0-tps-1366-486.jpg" alt="image" style="width:550px;height:180px;">
</div>

#### 多模态大模型协作系统-HuggingGPT
 
目前大模型解决单一模态的问题已经涌现出很多方法了，但是还无法处理复杂的人工智能任务如多模态内容的生成。  

LLM具有强大的语言能力，HuggingFace具有丰富的人工智能模型，如果能够将这两种能力结合起来，那就能够处理复杂的任务了；浙江大学联合微软亚洲研究院提出了HuggingGPT，它能够覆盖不同模态和领域的众多复杂人工智能任务，在语言、视觉、语音和其他具有挑战性的任务方面取得令人印象深刻的成果，这为 AGI 铺平了新的道路。

<div align="center">
<img src="https://img.alicdn.com/imgextra/i3/O1CN01txRxZ01r8RQCEESKO_!!6000000005586-0-tps-1082-480.jpg" alt="image" style="width:600px;height:280px;">
</div>

HuggingGPT 的工作可以分为四个阶段：

- 任务规划：使用 LLM 分析用户请求，将其分解为多个子任务，规划任务顺序和依赖关系。

- 模型选择：对于子任务，LLM 将根据模型描述来调用 HuggingFace 上的专家模型。

- 任务执行：每个专家模型执行所分配的子任务，返回执行结果。

- 响应生成：最后由 LLM 集成所有专家模型的结果，并为用户生成答案。


## ✅ 本节小结
通过本小节内容的学习，你已经学会了如何使用如何构建插件/工具来构建Agent应用了。

## 🔥 课后小测验

【多选题】2.5.1. 以下哪些代码片段可以正确创建一个使用夸克搜索的助手？( )

A. dashscope.Assistants.create(model='qwen-max', tools=['quark_search'])

B. dashscope.Assistants.create(model='qwen-max', tools=[{'type': 'quark_search'}])

C. dashscope.Assistants.create(model='qwen-max', tools={'quark_search'})

D. dashscope.Assistants.create(model='qwen-max', tool='quark_search')

E. dashscope.Assistants.create(model='qwen-max', tools=[{'name': 'quark_search'}])

F. dashscope.Assistants.create(model='qwen-max', tools=[{'type': 'quark_search', 'params': {}}])

答案：B, F (虽然B是示例代码，F更完整，params 可为空对象，表示使用默认参数)

<br>

【多选题】2.5.2. 以下示例代码中，query 方法是如何处理用户查询的？（ ）
```python
def query(self, query:str):
    '''
    query: string, the query string to the assistant. e.g. Who is the Jack Chou ?
    '''
    message = dashscope.Messages.create(self.thread.id, content=query)
    message_run = dashscope.Runs.create(self.thread.id, assistant_id=self.assistant.id)
    run_status = dashscope.Runs.wait(message_run.id, thread_id=self.thread.id)
    if run_status.required_action:
        self._forward_and_submit_outputs(run_status)
        run_status = dashscope.Runs.wait(run_status.id, thread_id=self.thread.id)
        
    msgs = dashscope.Messages.list(self.thread.id)
    answer = json.loads(json.dumps(msgs, default=lambda o: o.__dict__))['data'][0]['content'][0]['text']['value']
    return answer

```


A. 创建一个新的消息，并将用户查询作为消息内容。

B. 启动一个新的运行实例，并关联到指定的 Assistant。

C. 等待运行实例完成，并获取结果。

D. 如果运行实例需要调用函数，则调用 _forward_and_submit_outputs 方法。

E. 直接返回大语言模型生成的文本，无需处理。

F. 从消息列表中提取最终的答案文本。

答案：A, B, C, D, F